In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
import json
import re
import csv

In [4]:
pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 236.6 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
# CREATING A CSV FILE
house_data_csv = open('house_data.csv', 'a')

csv_writer = csv.writer(house_data_csv)


In [24]:
for i in range(615,631,1):
    time.sleep(1)
    api_key='aa6a98d769960d77d7a5fb7dcb1f0084'
    url = f"http://api.scraperapi.com?api_key={api_key}&url=https://propertypro.ng/property-for-rent?page={i}"

    response = requests.get(url)

    if response.status_code == 200:
        print("Connection succesful...")
    else:
        print(f"Connection error {ConnectionError} ")
        
    try:
        soup = BeautifulSoup(response.text, 'lxml')
        scripts = soup.find_all("script", type="application/ld+json")

        if not scripts:
            print(f"⚠️ Script tag not found on page {i}, skipping...")
            continue  # Skip pages without JSON data

        # Use the first script tag (if multiple exist)
        script_tag = scripts[0]
        parsed_data = json.loads(script_tag.text)

        house_raw_data = parsed_data.get('itemListElement', [])
        
        
        print(f"✅ Successfully scraped page {i}, found {len(house_raw_data)} properties.")
        
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Network error on page {i}: {e}, retrying in 10 seconds...")
        time.sleep(10)  # Wait and retry
        continue
    
    except (json.JSONDecodeError, IndexError, KeyError) as e:
        print(f"⚠️ Error parsing JSON on page {i}: {e}")
        continue  # Skip this page


    for data in house_raw_data :
        try:
            house_info = data["name"].split('|')
            url_link = data['url']

            print(f"🔗 Scraping property: {url_link}")

            url = f"http://api.scraperapi.com?api_key={api_key}&url={url_link}"
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            soup_2 = BeautifulSoup(response.text, 'lxml')
            scripts_2 = soup_2.find_all("script", type='application/ld+json')

            if len(scripts_2) < 2:
                print(f"⚠️ Skipping property {url_link}, script tag missing.")
                continue

            parsed_data_2 = json.loads(scripts_2[1].text)
            location_info = parsed_data_2.get("address", {})

            state = location_info.get("addressLocality", None)
            city = location_info.get("addressRegion", None)

            print(f"🏡 Property in {city}, {state}")

        except Exception as e:
            print(f"⚠️ Skipping property {url_link} due to error: {e}")
            continue  # Skip this property and continue
        
        # Getting features div from html 
        features_container = soup_2.find_all("div", class_='amen-grid')

        extra = None
        constant_electricity = "No"
        parking_lot = "No"
        
        # GETTING ALL FEATURES
        if len(features_container) > 0:
            tag = features_container[0].find_all("a", href="#")
            for a in tag:
                feature_text = a.text.strip()  # Extract text from <a> tag
                extra = len(feature_text)
                
                # Check if "electricity" is in the feature text (case-insensitive)
                if "electricity" in feature_text.casefold():  
                    constant_electricity = "Yes"
                elif "parking" in feature_text.casefold():
                    parking_lot = "Yes"
        else:
            extra = None
            constant_electricity = "No"
            
        
        # GETTING THE BEDROOMS, BATHROOMS AND TOILET NUMBER
        bedrooms = bathrooms = toilets = ""
        
        rooms_container = soup_2.find_all("div", class_='property-pros')
        if len(rooms_container) == 0:
            bedrooms = bathrooms = toilets = None
        else:
            rooms_container = soup_2.find_all("div", class_='property-pros')[0]
            uls = rooms_container.find_all("ul")
            # Loop through <ul> and find <li> elements
            for ul in uls:
                lis = ul.find_all("li")
                for li in lis:
                    text = li.get_text(strip=True)  # Get the text inside <li>
                    
                    if "Beds" in text:
                        bedrooms = text.split(" ")[0]  # Extract number before "Beds"
                    elif "Baths" in text:
                        bathrooms = text.split(" ")[0]  # Extract number before "Baths"
                    elif "Toilets" in text:
                        toilets = text.split(" ")[0]  # Extract number before "Toilets"
                    
                
        # GETTING PRICE FROM DESCRIPTION
        price_filter = data["description"].split(".")[1].split("at")[-1]
        
        # GETTING THE APARTMENT TYPE FROM DESCRIPTION
        title_filter = data["description"]
        
        
        # GETTING THE SERVICE FROM  DESCRIPTION
        serviced_filter = data["description"]

    
        def extract_serviced_houses(serviced_house):
            serviced = "No"
            if "Serviced" in serviced_house:
                serviced = "Yes"
            
            return serviced
        

        
        def extract_apartment_type(apartment_type):
            if "for rent in" in apartment_type and "Office" not in apartment_type:
                apartment_type_split = apartment_type.split("for rent in")[0]
            
            # Check if "bedroom" exists before splitting
                if "bedroom" in apartment_type_split:
                    apartment_type_split_2 = apartment_type_split.split("bedroom")
                    if len(apartment_type_split_2) > 1:
                        apartment_type_cleaned = (apartment_type_split_2[1].replace(",", "").replace("House", "").strip())
                        return apartment_type_cleaned
                    else:
                        return None  # "bedroom" is at the end with no type after it
                else:
                    return None  # "bedroom" is missing in the string
            else:
                return None 
                
        # # Getting all Values from calling functions 

        
        # state, city = extract_state(data["description"]) or (None, None) # Location
        
        serviced_house = extract_serviced_houses(serviced_filter) # Serviced "YES" or "NO"
        
        title = extract_apartment_type(title_filter)
        
        house_info_dict = {
            'bedroom' : bedrooms,
            'parking_lot' : parking_lot,
            'bathroom' : bathrooms,
            'toilet' : toilets,
            'town' : city,
            'state' : state,
            'serviced' : serviced_house,
            "extras" : extra,
            "Stable Electricity" : constant_electricity,
            "title" : title,
            "price" : price_filter,
        }
        house_info_dict
        print(f"{house_info_dict["title"]} in {house_info_dict["town"]} costs {house_info_dict["price"]} for rent")
        csv_writer.writerow([house_info_dict["bedroom"], house_info_dict["parking_lot"], house_info_dict["bathroom"] , house_info_dict["toilet"] , house_info_dict["town"] , house_info_dict["state"] , house_info_dict["serviced"] , house_info_dict["extras"] , house_info_dict["Stable Electricity"] , house_info_dict["title"], house_info_dict["price"] ])
house_data_csv.close()
   
      

Connection succesful...
✅ Successfully scraped page 615, found 1 properties.
🔗 Scraping property: https://propertypro.ng/property/2-bedroom-house-for-rent-lekki-phase-1-lekki-lagos-9MGUU
🏡 Property in Lekki, Lagos
Blocks Of Flats in Lekki costs  ₦13,000,000/year for rent
Connection succesful...
⚠️ Script tag not found on page 616, skipping...
Connection succesful...
⚠️ Script tag not found on page 617, skipping...


KeyboardInterrupt: 

In [171]:
'''
 Try and split with "for rent in" then take the first list.
 split again with house, if house, if not, split with comma.
 then split with bedroom and take thre second index
''' 

'\n Try and split with "for rent in" then take the first list.\n split again with house, if house, if not, split with comma.\n then split with bedroom and take thre second index\n'

In [278]:
        
        # def price_conversion(price):  
        #         if str(price).startswith("$"):
        #             value = price[1:].replace(",","")
        #             if value.isdigit(): 
        #                 naira_value = f"₦{int(value) * 1610}"
        #                 return naira_value
        #         else:
        #             value = price[1:].replace(",","")
        #             naira_value = value
        #             return naira_value